In [17]:
import xarray as xr
import numpy as np
import os
from metpy import calc as mpcalc

In [23]:
inputdir = "/run/media/ludo/DATA/google-drive/Thèse/EUREC4a/github/Input/"
dropsonde_path = os.path.join(inputdir, "Dropsondes", "all_dropsondes.nc")
radiosonde_path = os.path.join(inputdir, "Radiosondes", "all_radiosondes.nc")

# inputdir = '/Users/bfildier/Data/EUREC4A/merged/sondes'
# radiosondes = xr.open_dataset(os.path.join(inputdir,'all_radiosondes.nc'))
# dropsondes = xr.open_dataset(os.path.join(inputdir,'all_dropsondes.nc'))

In [72]:
def prepare_dropsondes(dropsonde_path):
    dropsondes = xr.open_dataset(dropsonde_path)

    dropsondes = dropsondes.drop_vars(["trajectory","w_wind", "dz", "vt", "alt", "reference_time", \
                                   "reference_pres", "reference_tdry", "reference_rh", "reference_wspd",\
                                 "reference_wdir", "reference_lat", "reference_lon", "reference_alt", "T_interp",\
                                 "T", "rh_interp", "q"])

    dropsondes["tdry"].attrs["units"] = "degC"
    dropsondes["dp"].attrs["units"] = "degC"
    dropsondes["pres"].attrs["units"] = "hPa"

    dropsondes["ascent_flag"] = (["launch_time"], np.zeros(len(dropsondes.launch_time.values)))
    
    return dropsondes

In [73]:
def prepare_radiosondes(radiosonde_path)

    radiosondes = xr.open_dataset(radiosonde_path)

    radiosondes = radiosondes.reset_coords(['lon','lat','time'])

    radiosondes["tdry"].attrs["units"] = "degC"
    radiosondes["pres"].attrs["units"] = "hPa"
    radiosondes["dp"].attrs["units"] = "degC"


    theta = mpcalc.potential_temperature(radiosondes["pres"], radiosondes["tdry"])
    radiosondes["theta"] = (["launch_time", "gpsalt"], theta.magnitude)

    theta_v = mpcalc.virtual_potential_temperature(radiosondes["pres"], radiosondes["tdry"], radiosondes["mr"]/1000)
    radiosondes["theta_v"] = (["launch_time", "gpsalt"], theta_v.magnitude)

    theta_e = mpcalc.equivalent_potential_temperature(radiosondes["pres"], radiosondes["tdry"], radiosondes["dp"])
    radiosondes["theta_e"] = (["launch_time", "gpsalt"], theta_e.magnitude)

    del radiosondes.attrs["title"]
    del radiosondes.attrs["doi"]
    del radiosondes.attrs["surface_altitude"]
    del radiosondes.attrs["featureType"]
    del radiosondes.attrs["Conventions"]
    del radiosondes.attrs["history"]
    del radiosondes.attrs["NCO"]
    del radiosondes.attrs["nco_openmp_thread_number"]

    radiosondes = radiosondes.drop_vars(["specific_humidity", "ascent_rate"])

    length = len(radiosondes.launch_time.values)
    list_str = []
    for i in range(length):
        radiosonde = radiosondes.isel(launch_time = i)

        if (radiosonde.platform.values==1):
            list_str.append("BCO")
        elif (radiosonde.platform.values==2):
            list_str.append("BCO")
        elif (radiosonde.platform.values==3):
            list_str.append("RHB")
        elif (radiosonde.platform.values==4):
            list_str.append("MER")
        else:
            list_str.append("ATL")     

    array = np.array(list_str, dtype=object)
    radiosondes["platform"] = (["launch_time"], array)
    
    return radiosondes

<xarray.Dataset>
Dimensions:      (gpsalt: 3000, launch_time: 1454)
Coordinates:
  * gpsalt       (gpsalt) float32 0.0 10.0 20.0 30.0 ... 29970.0 29980.0 29990.0
  * launch_time  (launch_time) datetime64[ns] 2020-01-16T15:14:58 ... 2020-02-16T04:14:33
Data variables:
    tdry         (launch_time, gpsalt) float32 nan nan nan nan ... nan nan nan
    dp           (launch_time, gpsalt) float32 nan nan nan nan ... nan nan nan
    time         (launch_time, gpsalt) datetime64[ns] ...
    wspd         (launch_time, gpsalt) float32 ...
    pres         (launch_time, gpsalt) float32 nan nan nan nan ... nan nan nan
    u_wind       (launch_time, gpsalt) float32 ...
    v_wind       (launch_time, gpsalt) float32 ...
    lat          (launch_time, gpsalt) float32 ...
    lon          (launch_time, gpsalt) float32 ...
    mr           (launch_time, gpsalt) float32 nan nan nan nan ... nan nan nan
    wdir         (launch_time, gpsalt) float32 ...
    rh           (launch_time, gpsalt) float32 ...
 

In [74]:
dropsondes = prepare_dropsondes(dropsonde_path)
radiosondes = prepare_radiosondes(radiosonde_path)
print(dropsonde, radiosonde)

<xarray.Dataset>
Dimensions:      (gpsalt: 1001, launch_time: 1170)
Coordinates:
  * launch_time  (launch_time) datetime64[ns] 2020-01-15T15:19:15 ... 2020-02-18T16:18:02
  * gpsalt       (gpsalt) int64 0 10 20 30 40 50 ... 9960 9970 9980 9990 10000
Data variables:
    time         (launch_time, gpsalt) float64 ...
    pres         (launch_time, gpsalt) float64 ...
    tdry         (launch_time, gpsalt) float64 ...
    dp           (launch_time, gpsalt) float64 ...
    rh           (launch_time, gpsalt) float64 ...
    u_wind       (launch_time, gpsalt) float64 ...
    v_wind       (launch_time, gpsalt) float64 ...
    wspd         (launch_time, gpsalt) float64 ...
    wdir         (launch_time, gpsalt) float64 ...
    mr           (launch_time, gpsalt) float64 ...
    theta        (launch_time, gpsalt) float64 ...
    theta_e      (launch_time, gpsalt) float64 ...
    theta_v      (launch_time, gpsalt) float64 ...
    lat          (launch_time, gpsalt) float64 ...
    lon          (la

In [ ]:
all_sondes = xr.concat([dropsondes,radiosondes],dim='launch_time')

In [ ]:
print(all_sondes)

In [ ]:
all_sondes.to_netcdf(os.path.join(inputdir, "all_sondes.nc"))